Übung: kubectl-CLI
----------------------------

Das `kubectl`-Kommando stellt, eine der Schaltzentralen des K8s Clusters zur Administration der Ressourcen dar.

In dieser Übung verwenden wir das `kubectl`-Kommando zur Erstellen eines Pod's, Jobs und eines Deployment's.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace test

Erzeugen eines Pod's 

In [ ]:
! kubectl run nginxpod1 --image=nginx --restart=Never -n test

Ausgabe der Erzeugten Ergebnisse und die YAML Datei welche den Pod beschreibt:

In [ ]:
! kubectl get all -n test

In [ ]:
! kubectl get pod nginxpod1 -o yaml -n test | head -19

Nun Erzeugen wir ein Deployment welches automatisch einen Pod erzeugt:

In [ ]:
! kubectl run --image=nginx nginxdep1 -n test

Ausgabe der Erzeugten Ergebnisse und die YAML Datei welche den Pod beschreibt.

Es sollten jetzt ein Deployment, ein ReplicaSet und zwei Pod's vorhanden sein.

In [ ]:
! kubectl get all -n test

In [ ]:
! kubectl get deploy nginxdep1 -o yaml -n test 

Zum Aufräumen genügt es, dass Namespace zu löschen

In [ ]:
! kubectl delete namespace test